In [1]:
import pandas as pd
import os
from itertools import product
import numpy as np

In [2]:
orders = ['LO', 'NLO', 'N2LO', 'N3LO']
Lambdas = [450, 500]

high_density = True

if high_density:
    bodies = ['NN+3N']
    xs = ['0.5']
else:
    bodies = ['NN-only', 'NN+3N']
    xs = ['0.', '0.5']
fits = {
    (450, 'LO'): [1],
    (500, 'LO'): [6],
    (450, 'NLO'): [2],
    (500, 'NLO'): [7],
    (450, 'N2LO'): [1, 2, 3],
    (500, 'N2LO'): [4, 5, 6],
    (450, 'N3LO'): [7, 8, 9],
    (500, 'N3LO'): [10, 11, 12],
}
data_dir = os.path.abspath('../raw_data/')


i = 0
dfs = []

for order, Lambda, body, x in product(orders, Lambdas, bodies, xs):
    if body == 'NN-only':
        filename = os.path.join(data_dir, f'{body}/EOS_x_{x}_{order}_EM{Lambda}new.txt')
        i += 1
        df = pd.read_csv(filename, sep='\t', header=0)
        df.rename(columns={'# kf': 'kf', 'HF_NN': 'MBPT_HF', 'Scnd_NN': 'MBPT_2',
                           'Trd_NN_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
        df['Lambda'] = Lambda
        df['OrderEFT'] = order
        df['Body'] = body
        df['x'] = float(x)
        df['fit'] = np.nan
        dfs.append(df)
    else:
        for fit in fits[Lambda, order]:
            if high_density:
                filename = f'{body}_high_density/SNM/dat_high_den_Ham_{fit}_{order}_EM{Lambda}new.dat'
            else:
                filename = f'{body}/EOS_x_{x}_Ham_{fit}_{order}_EM{Lambda}new.txt'
            filename = os.path.join(data_dir, filename)
            i += 1
            if high_density:
                df = pd.read_csv(
                    filename, sep='\t', header=None,
                    names=['kf', 'n', 'MBPT_4', '??', '???']
                )
            else:
                df = pd.read_csv(filename, sep='\t', header=0)
                df.rename(columns={'# kf': 'kf', 'HF_tot': 'MBPT_HF', 'Scnd_tot': 'MBPT_2',
                                   'Trd_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
            df['Lambda'] = Lambda
            df['OrderEFT'] = order
            df['Body'] = body
            df['x'] = float(x)
            if order in ['LO', 'NLO']:
                fit = np.nan
            df['fit'] = fit
            dfs.append(df)
            
            # Copy data for easier parsing later on. Given LO and NLO the same fit #s at N2LO and N3LO.
            # if order in ['LO', 'NLO']:
            #     for other_fit in [*fits[Lambda, 'N2LO'], *fits[Lambda, 'N3LO']]:
            #         if other_fit != fit:
            #             df = df.copy() # just in case it would mess with stuff already in dfs
            #             df['fit'] = other_fit
            #             dfs.append(df)
            
    

print('Num files read:', i)
df = pd.concat(dfs, ignore_index=True)

# filename1 = '/Users/jordanmelendez/Dropbox/Research/Matter/Matter_Convergence_Analysis/data/NN-only/EOS_x_0._LO_EM500new.txt'
# filename2 = '/Users/jordanmelendez/Dropbox/Research/Matter/Matter_Convergence_Analysis/data/NN+3N/EOS_x_0._Ham_4_N2LO_EM500new.txt'
# df1 = pd.read_csv(filename1, sep='\t', header=0)
# df1.rename(columns={'# kf': 'kf', 'HF_NN': 'MBPT_HF', 'Scnd_NN': 'MBPT_2',
#                     'Trd_NN_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
# df2 = pd.read_csv(filename2, sep='\t', header=0)
# df2.rename(columns={'# kf': 'kf', 'HF_tot': 'MBPT_HF', 'Scnd_tot': 'MBPT_2',
#                     'Trd_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)

Num files read: 16


In [3]:
df.dtypes

kf          float64
n           float64
MBPT_4      float64
??          float64
???         float64
Lambda        int64
OrderEFT     object
Body         object
x           float64
fit         float64
dtype: object

In [4]:
df['fit'].unique()

array([nan,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [5]:
df[df['OrderEFT'] == 'LO']

,kf,n,MBPT_4,??,???,Lambda,OrderEFT,Body,x,fit
0,0.904594,0.05,-7.447576,-0.120308,-0.004312,450,LO,NN+3N,0.5,NaN
1,0.961274,0.06,-7.912989,-0.161974,-0.004643,450,LO,NN+3N,0.5,NaN
2,1.011959,0.07,-8.348165,-0.206197,-0.004875,450,LO,NN+3N,0.5,NaN
3,1.058019,0.08,-8.755395,-0.252188,-0.005034,450,LO,NN+3N,0.5,NaN
4,1.100385,0.09,-9.137030,-0.299383,-0.005144,450,LO,NN+3N,0.5,NaN
5,1.139717,0.10,-9.495355,-0.347404,-0.005221,450,LO,NN+3N,0.5,NaN
6,1.176507,0.11,-9.832536,-0.396025,-0.005281,450,LO,NN+3N,0.5,NaN
7,1.211130,0.12,-10.150590,-0.445141,-0.005335,450,LO,NN+3N,0.5,NaN
8,1.243879,0.13,-10.451384,-0.494743,-0.005393,450,LO,NN+3N,0.5,NaN
9,1.274989,0.14,-10.736631,-0.544899,-0.005460,450,LO,NN+3N,0.5,NaN


In [6]:

savename = 'all_matter_data.csv'
if high_density:
    savename = 'all_matter_data_high_density.csv'
df.to_csv(savename, index=False)